In [1]:
import pandas as pd
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.text import Tokenizer

data=pd.read_csv("/kaggle/input/tamil-troll/train_captions.csv")
data.head()


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

,Unnamed: 0,imagename,captions
0,0,Not_troll_0.jpg,Ada pikkalipayalugala
1,1,Not_troll_1.jpg,Etho sambavam nadandhirukkum pola
2,2,Not_troll_10.jpg,Vunnayellam frienda vechirukken paaru
3,3,Not_troll_100.jpg,Idho! Ivan dhaan naan nasama ponadhukku kaaranam
4,4,Not_troll_1000.jpg,Ennada lusu thanama pesikittu irukka lusu payale


In [2]:
data2=pd.read_csv("/kaggle/input/tamil-troll/test_captions.csv")

In [3]:
# setting labels
def labeling(train):
  train['Label'] = train['imagename'].str.replace('\d+', '')
  train['Label'] = train['Label'].str.replace('_.jpg', '')
  train['Label'] = train['Label'].str.replace('_.png', '')
  return train['Label']

data['Label']= labeling(data)

data2.head()

/tmp/ipykernel_28/2200995685.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Label'] = train['imagename'].str.replace('\d+', '')
/tmp/ipykernel_28/2200995685.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Label'] = train['Label'].str.replace('_.jpg', '')
/tmp/ipykernel_28/2200995685.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Label'] = train['Label'].str.replace('_.png', '')


,Unnamed: 0,imagename,captions,label
0,0,test_img_0.jpg,sugarkaga nadandhava vida figuregaga nadandhav...,troll
1,1,test_img_1.jpg,i have come for my stones stones thaane.....,troll
2,2,test_img_2.jpg,"""special porotta"" nu pottuierukke spacial kum ...",troll
3,3,test_img_3.jpg,*we : amma .. cooker 3 whistle vanthuchu off p...,troll
4,4,test_img_4.jpg,creating whatsapp group - 1st day vaanga ji.. ...,troll


In [4]:
data2['label']=data2['label'].apply(lambda x: 1 if x=='not_troll' else 0)
data2

,Unnamed: 0,imagename,captions,label
0,0,test_img_0.jpg,sugarkaga nadandhava vida figuregaga nadandhav...,0
1,1,test_img_1.jpg,i have come for my stones stones thaane.....,0
2,2,test_img_2.jpg,"""special porotta"" nu pottuierukke spacial kum ...",0
3,3,test_img_3.jpg,*we : amma .. cooker 3 whistle vanthuchu off p...,0
4,4,test_img_4.jpg,creating whatsapp group - 1st day vaanga ji.. ...,0
...,...,...,...,...
662,662,test_img_662.jpg,DURING SCHOOL DAYS ME & FRNDS: ANNA MAIL CHEC...,0
663,663,test_img_663.jpg,REMEMBER THIS KID THIS KID IS RIGHT NOW,0
664,664,test_img_664.jpg,*SINGLES: IVAN VERA ENGALA ROMBA TORCHER PANDR...,0
665,665,test_img_665.jpg,ENAIKI PUBG LA CHICKEN DINNER VANGURA PAYALUGA...,1


In [5]:
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()

# function for encoding labels
def encodelabels(labels):  
  labels =  Encoder.fit_transform(labels)                                                      
  return labels

data['Label'] = encodelabels(data['Label'])


In [6]:
data2.head(10)


,Unnamed: 0,imagename,captions,label
0,0,test_img_0.jpg,sugarkaga nadandhava vida figuregaga nadandhav...,0
1,1,test_img_1.jpg,i have come for my stones stones thaane.....,0
2,2,test_img_2.jpg,"""special porotta"" nu pottuierukke spacial kum ...",0
3,3,test_img_3.jpg,*we : amma .. cooker 3 whistle vanthuchu off p...,0
4,4,test_img_4.jpg,creating whatsapp group - 1st day vaanga ji.. ...,0
5,5,test_img_5.jpg,*FOODIE : YAARUM PAARKKAMAL ENNAI PAARKIREN EN...,0
6,6,test_img_6.jpg,MARGALI MAADHA EFFECT YENNA KULLURU,0
7,7,test_img_7.jpg,YENNAMMA FEEL PANNI YELUTHI ERUKRANDA AVAN,0
8,8,test_img_8.jpg,TAMIL SERIALS BE LIKE... INI AVARUKKU BADHIL.....,1
9,9,test_img_9.jpg,Mama ponnu Expectation reality,0


In [7]:
data['Label'].value_counts()

1    1282
0    1018
Name: Label, dtype: int64

In [8]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [9]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [10]:
# # Bert layers
# text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
# preprocessed_text = bert_preprocess(text_input)
# outputs = bert_encoder(preprocessed_text)

# # Extract the intermediate layer representation
# intermediate_layer_output = outputs['pooled_output']

# # Create a new model to get the feature vectors
# feature_extraction_model = tf.keras.Model(inputs=text_input, outputs=intermediate_layer_output)

# # Use the feature extraction model to get the feature vectors
# bert1 = feature_extraction_model.predict(data['captions'])

In [11]:
# # Print the shape of the feature vectors
# print("Shape of feature vectors:", bert1.shape)

# # Print the length of the feature vectors
# print("Length of feature vectors:", len(bert1))

# # Print the first 10 values of the feature vectors
# print("First 1 values of feature vectors:")
# print(bert1[:1])


In [12]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1024, activation='relu', name="feature")(l)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [13]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [14]:
model.fit(data['captions'], data['Label'], epochs=10)

Epoch 1/10
72/72 [==============================] - 38s 316ms/step - loss: 0.6201 - accuracy: 0.7200 - precision: 0.7406 - recall: 0.7660
Epoch 2/10
72/72 [==============================] - 24s 327ms/step - loss: 0.4848 - accuracy: 0.7722 - precision: 0.7929 - recall: 0.8003
Epoch 3/10
72/72 [==============================] - 24s 334ms/step - loss: 0.4577 - accuracy: 0.7843 - precision: 0.8023 - recall: 0.8136
Epoch 4/10
72/72 [==============================] - 25s 348ms/step - loss: 0.4197 - accuracy: 0.8117 - precision: 0.8361 - recall: 0.8237
Epoch 5/10
72/72 [==============================] - 25s 353ms/step - loss: 0.4222 - accuracy: 0.8122 - precision: 0.8269 - recall: 0.8385
Epoch 6/10
72/72 [==============================] - 25s 348ms/step - loss: 0.4258 - accuracy: 0.8052 - precision: 0.8263 - recall: 0.8237
Epoch 7/10
72/72 [==============================] - 25s 348ms/step - loss: 0.4327 - accuracy: 0.8061 - precision: 0.8261 - recall: 0.8261
Epoch 8/10
72/72 [================

In [15]:
# Evaluate the model on the test data
results = model.evaluate(data2['captions'], data2['label'])

# Extract the evaluation metrics
accuracy = results[model.metrics_names.index('accuracy')]
precision = results[model.metrics_names.index('precision')]
recall = results[model.metrics_names.index('recall')]

# Calculate the F1 score
f1_score = 2 * (precision * recall) / (precision + recall)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


21/21 [==============================] - 8s 352ms/step - loss: 1.4630 - accuracy: 0.4198 - precision: 0.3964 - recall: 0.8088
Accuracy: 0.419790118932724
Precision: 0.3963963985443115
Recall: 0.8088235259056091
F1 Score: 0.532043532010522


In [16]:
from sklearn.metrics import classification_report

# Make predictions on the test data
predictions = model.predict(data2['captions'])
predicted_labels = np.argmax(predictions, axis=1)

# Obtain true labels
true_labels = data2['label']

# Generate classification report
report = classification_report(true_labels, predicted_labels)

# Print the classification report
print(report)


21/21 [==============================] - 8s 344ms/step
              precision    recall  f1-score   support

           0       0.59      1.00      0.74       395
           1       0.00      0.00      0.00       272

    accuracy                           0.59       667
   macro avg       0.30      0.50      0.37       667
weighted avg       0.35      0.59      0.44       667



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
# Extract the feature vectors from the trained model
feature_extraction_model = tf.keras.Model(inputs=model.input,
                                          outputs=model.get_layer('feature').output)

# Get the feature vectors for your input data
bert2 = feature_extraction_model.predict(data['captions'])

72/72 [==============================] - 25s 345ms/step


In [47]:
# Print the shape of the feature vectors
print("Shape of feature vectors:", bert2.shape)

# Print the length of the feature vectors
print("Length of feature vectors:", len(bert2))

# Print the first 10 values of the feature vectors
print("First 10 values of feature vectors:")
print(bert2[:1])


Shape of feature vectors: (2300, 1024)
Length of feature vectors: 2300
First 10 values of feature vectors:
[[0. 0. 0. ... 0. 0. 0.]]


In [58]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1024, activation='relu', name="feature")(l)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [59]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [60]:
model.fit(data2['captions'], data2['label'], epochs=10)

Epoch 1/10
21/21 [==============================] - 10s 344ms/step - loss: 1.2190 - accuracy: 0.4978 - precision: 0.3507 - recall: 0.2721
Epoch 2/10
21/21 [==============================] - 7s 354ms/step - loss: 0.7177 - accuracy: 0.5337 - precision: 0.4163 - recall: 0.3566
Epoch 3/10
21/21 [==============================] - 8s 372ms/step - loss: 0.6947 - accuracy: 0.5652 - precision: 0.4541 - recall: 0.3272
Epoch 4/10
21/21 [==============================] - 8s 366ms/step - loss: 0.6738 - accuracy: 0.5967 - precision: 0.5118 - recall: 0.2390
Epoch 5/10
21/21 [==============================] - 8s 361ms/step - loss: 0.7069 - accuracy: 0.5502 - precision: 0.4417 - recall: 0.3897
Epoch 6/10
21/21 [==============================] - 7s 353ms/step - loss: 0.6840 - accuracy: 0.5982 - precision: 0.5192 - recall: 0.1985
Epoch 7/10
21/21 [==============================] - 8s 360ms/step - loss: 0.7061 - accuracy: 0.5847 - precision: 0.4850 - recall: 0.2978
Epoch 8/10
21/21 [======================

In [61]:
# Extract the feature vectors from the trained model
feature_extraction_model = tf.keras.Model(inputs=model.input,
                                          outputs=model.get_layer('feature').output)

# Get the feature vectors for your input data
bert_test = feature_extraction_model.predict(data2['captions'])

21/21 [==============================] - 8s 333ms/step


In [62]:
# Print the shape of the feature vectors
print("Shape of feature vectors:", bert_test.shape)

# Print the length of the feature vectors
print("Length of feature vectors:", len(bert_test))

# Print the first 10 values of the feature vectors
print("First 10 values of feature vectors:")
print(bert_test[:1])


Shape of feature vectors: (667, 1024)
Length of feature vectors: 667
First 10 values of feature vectors:
[[0.37655833 0.         0.         ... 0.63003796 0.         0.        ]]


In [17]:
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.text import Tokenizer
# Setting seed for reproducibiltiy
SEED = 42
keras.utils.set_random_seed(SEED)

In [18]:
# DATA
BUFFER_SIZE = 512
BATCH_SIZE = 256

# AUGMENTATION
IMAGE_SIZE=72
PATCH_SIZE = 6
NUM_PATCHES = (IMAGE_SIZE // PATCH_SIZE) ** 2

# OPTIMIZER
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001

# TRAINING
EPOCHS = 10

# ARCHITECTURE
LAYER_NORM_EPS = 1e-6
TRANSFORMER_LAYERS = 8
PROJECTION_DIM = 64
NUM_HEADS = 4
TRANSFORMER_UNITS = [
    PROJECTION_DIM * 2,
    PROJECTION_DIM,
]
MLP_HEAD_UNITS = [2048, 1024]

In [19]:
# training input in array
img_size = 72
train_image_dir = '/kaggle/input/tamil-troll/uploaded_tamil_memes 1/uploaded_tamil_memes 1'
# train_image_array = create_dataset(train_image_dir)
# train_image_array.shape

train_image_files = os.listdir(train_image_dir)


# Create an empty array to store the images
train_images_array = np.zeros((len(train_image_files), img_size, img_size, 3), dtype=np.uint8)



# Loop through the training images and load them into the array
for i, file in enumerate(train_image_files):
    img = cv2.imread(os.path.join(train_image_dir, file))
    img = cv2.resize(img, (img_size, img_size))
    train_images_array[i] = img

In [20]:
# training input in array
# img_size = 224
test_image_dir = '/kaggle/input/tamil-troll/uploaded_tamil_memes 1/test_img'
# train_image_array = create_dataset(train_image_dir)
# train_image_array.shape

test_image_files = os.listdir(test_image_dir)


# Create an empty array to store the images
test_images_array = np.zeros((len(test_image_files), img_size, img_size, 3), dtype=np.uint8)



# Loop through the training images and load them into the array
for i, file in enumerate(test_image_files):
    img = cv2.imread(os.path.join(test_image_dir, file))
    img = cv2.resize(img, (img_size, img_size))
    test_images_array[i] = img

In [21]:
print(len(train_images_array))
print(len(test_images_array))

2300
667


In [22]:
x_train=train_images_array
x_test=test_images_array
y_train=data['Label']
y_test=data2['label']

In [23]:
x_test

array([[[[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        ...,

        [[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1]],

        [[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1]],

        [[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  1,   1,   1],
        

In [24]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(72, 72),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)


In [25]:
# Assuming you have a batch of images x_train with shape (batch_size, height, width, channels)

# Pass the sample batch through the data_augmentation model
augmented_images = data_augmentation.predict(x_train)

# Check the shape of augmented_images
num_images_after_augmentation = augmented_images.shape[0]
print("Number of images after augmentation:", num_images_after_augmentation)


72/72 [==============================] - 0s 2ms/step
Number of images after augmentation: 2300


In [31]:
class ShiftedPatchTokenization(layers.Layer):
    def __init__(
        self,
        image_size=IMAGE_SIZE,
        patch_size=PATCH_SIZE,
        num_patches=NUM_PATCHES,
        projection_dim=PROJECTION_DIM,
        vanilla=False,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.vanilla = vanilla  # Flag to swtich to vanilla patch extractor
        self.image_size = image_size
        self.patch_size = patch_size
        self.half_patch = patch_size // 2
        self.flatten_patches = layers.Reshape((num_patches, -1))
        self.projection = layers.Dense(units=projection_dim)
        self.layer_norm = layers.LayerNormalization(epsilon=LAYER_NORM_EPS)

    def crop_shift_pad(self, images, mode):
        # Build the diagonally shifted images
        if mode == "left-up":
            crop_height = self.half_patch
            crop_width = self.half_patch
            shift_height = 0
            shift_width = 0
        elif mode == "left-down":
            crop_height = 0
            crop_width = self.half_patch
            shift_height = self.half_patch
            shift_width = 0
        elif mode == "right-up":
            crop_height = self.half_patch
            crop_width = 0
            shift_height = 0
            shift_width = self.half_patch
        else:
            crop_height = 0
            crop_width = 0
            shift_height = self.half_patch
            shift_width = self.half_patch

        # Crop the shifted images and pad them
        crop = tf.image.crop_to_bounding_box(
            images,
            offset_height=crop_height,
            offset_width=crop_width,
            target_height=self.image_size - self.half_patch,
            target_width=self.image_size - self.half_patch,
        )
        shift_pad = tf.image.pad_to_bounding_box(
            crop,
            offset_height=shift_height,
            offset_width=shift_width,
            target_height=self.image_size,
            target_width=self.image_size,
        )
        return shift_pad

    def call(self, images):
        if not self.vanilla:
            # Concat the shifted images with the original image
            images = tf.concat(
                [
                    images,
                    self.crop_shift_pad(images, mode="left-up"),
                    self.crop_shift_pad(images, mode="left-down"),
                    self.crop_shift_pad(images, mode="right-up"),
                    self.crop_shift_pad(images, mode="right-down"),
                ],
                axis=-1,
            )
        # Patchify the images and flatten it
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        flat_patches = self.flatten_patches(patches)
        if not self.vanilla:
            # Layer normalize the flat patches and linearly project it
            tokens = self.layer_norm(flat_patches)
            tokens = self.projection(tokens)
        else:
            # Linearly project the flat patches
            tokens = self.projection(flat_patches)
        return (tokens, patches)

# BERT Vectors - 768


In [32]:
class PatchEncoder(layers.Layer):
    def __init__(
        self, num_patches=NUM_PATCHES, projection_dim=PROJECTION_DIM, **kwargs
    ):
        super().__init__(**kwargs)
        self.num_patches = num_patches
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )
        self.positions = tf.range(start=0, limit=self.num_patches, delta=1)

    def call(self, encoded_patches):
        encoded_positions = self.position_embedding(self.positions)
        encoded_patches = encoded_patches + encoded_positions
        return encoded_patches

In [33]:
class MultiHeadAttentionLSA(tf.keras.layers.MultiHeadAttention):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # The trainable temperature term. The initial value is
        # the square root of the key dimension.
        self.tau = tf.Variable(math.sqrt(float(self._key_dim)), trainable=True)

    def _compute_attention(self, query, key, value, attention_mask=None, training=None):
        query = tf.multiply(query, 1.0 / self.tau)
        attention_scores = tf.einsum(self._dot_product_equation, key, query)
        attention_scores = self._masked_softmax(attention_scores, attention_mask)
        attention_scores_dropout = self._dropout_layer(
            attention_scores, training=training
        )
        attention_output = tf.einsum(
            self._combine_equation, attention_scores_dropout, value
        )
        return attention_output, attention_scores

In [34]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


# Build the diagonal attention mask
diag_attn_mask = 1 - tf.eye(NUM_PATCHES)
diag_attn_mask = tf.cast([diag_attn_mask], dtype=tf.int8)

In [35]:
def create_vit_classifier(vanilla=False):
    inputs = layers.Input(shape=INPUT_SHAPE)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    (tokens, _) = ShiftedPatchTokenization(vanilla=vanilla)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder()(tokens)

    # Create multiple layers of the Transformer block.
    for _ in range(TRANSFORMER_LAYERS):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        if not vanilla:
            attention_output = MultiHeadAttentionLSA(
                num_heads=NUM_HEADS, key_dim=PROJECTION_DIM, dropout=0.1
            )(x1, x1, attention_mask=diag_attn_mask)
        else:
            attention_output = layers.MultiHeadAttention(
                num_heads=NUM_HEADS, key_dim=PROJECTION_DIM, dropout=0.1
            )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=TRANSFORMER_UNITS, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    # Global average pooling
    pooled_representation = layers.GlobalAveragePooling1D()(representation)
    pooled_representation = layers.Dropout(0.5)(pooled_representation)
    # Add MLP.
    features = mlp(pooled_representation, hidden_units=MLP_HEAD_UNITS, dropout_rate=0.5)
    # Output feature vector
    feature_vector = features

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=feature_vector)
    return model


In [36]:
NUM_CLASSES = 2
INPUT_SHAPE = (72, 72, 3)

In [37]:
# Some code is taken from:
# https://www.kaggle.com/ashusma/training-rfcx-tensorflow-tpu-effnet-b2.
from tensorflow.keras.optimizers.schedules import LearningRateSchedule

class WarmUpCosine(LearningRateSchedule):
    def __init__(self, learning_rate_base, total_steps, warmup_learning_rate, warmup_steps):
        super().__init__()

        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.pi = tf.constant(np.pi)

    def __call__(self, step):
        if self.total_steps < self.warmup_steps:
            raise ValueError("Total_steps must be larger or equal to warmup_steps.")

        cos_annealed_lr = tf.cos(
            self.pi * (tf.cast(step, tf.float32) - self.warmup_steps) /
            float(self.total_steps - self.warmup_steps)
        )
        learning_rate = 0.5 * self.learning_rate_base * (1 + cos_annealed_lr)

        if self.warmup_steps > 0:
            if self.learning_rate_base < self.warmup_learning_rate:
                raise ValueError(
                    "Learning_rate_base must be larger or equal to warmup_learning_rate."
                )
            slope = (self.learning_rate_base - self.warmup_learning_rate) / self.warmup_steps
            warmup_rate = slope * tf.cast(step, tf.float32) + self.warmup_learning_rate
            learning_rate = tf.where(step < self.warmup_steps, warmup_rate, learning_rate)

        return tf.where(step > self.total_steps, 0.0, learning_rate, name="learning_rate")


# def run_experiment(model, x_train, y_train, x_test, y_test, LEARNING_RATE, WEIGHT_DECAY, BATCH_SIZE, EPOCHS):
#     total_steps = int((len(x_train) / BATCH_SIZE) * EPOCHS)
#     warmup_epoch_percentage = 0.10
#     warmup_steps = int(total_steps * warmup_epoch_percentage)
#     scheduled_lrs = WarmUpCosine(
#         learning_rate_base=LEARNING_RATE,
#         total_steps=total_steps,
#         warmup_learning_rate=0.0,
#         warmup_steps=warmup_steps,
#     )

#     optimizer = tfa.optimizers.AdamW(
#         learning_rate=scheduled_lrs,
#         weight_decay=WEIGHT_DECAY
#     )

#     model.compile(
#         optimizer=optimizer,
#         loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#         metrics=[
#             keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
#             keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
#         ],
#     )

#     history = model.fit(
#         x=x_train,
#         y=y_train,
#         batch_size=BATCH_SIZE,
#         epochs=EPOCHS,
#         validation_split=0.1,
#     )
#     _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")
#     print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

#     return history


# # # Run experiments with the vanilla ViT
# # vit = create_vit_classifier(vanilla=True)
# # history = run_experiment(vit, x_train, y_train, x_test, y_test, LEARNING_RATE, WEIGHT_DECAY, BATCH_SIZE, EPOCHS)

# # Run experiments with the Shifted Patch Tokenization and Locality Self Attention modified ViT
# vit_sl = create_vit_classifier(vanilla=False)
# history = run_experiment(vit_sl, x_train, y_train, x_test, y_test, LEARNING_RATE, WEIGHT_DECAY, BATCH_SIZE, EPOCHS)
def run_experiment(model, x_train, y_train, x_test, y_test, LEARNING_RATE, WEIGHT_DECAY, BATCH_SIZE, EPOCHS):
    total_steps = int((len(x_train) / BATCH_SIZE) * EPOCHS)
    warmup_epoch_percentage = 0.10
    warmup_steps = int(total_steps * warmup_epoch_percentage)
    scheduled_lrs = WarmUpCosine(
        learning_rate_base=LEARNING_RATE,
        total_steps=total_steps,
        warmup_learning_rate=0.0,
        warmup_steps=warmup_steps,
    )

    optimizer = tfa.optimizers.AdamW(
        learning_rate=scheduled_lrs,
        weight_decay=WEIGHT_DECAY
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_split=0.1,
    )
    
    # Get the feature vectors before running
    before_vectors = model.predict(x_train)
    
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")
    
    # Get the feature vectors after running
    after_vectors = model.predict(x_train)
    
    return history, before_vectors, after_vectors


# Create the model
vit_sl = create_vit_classifier(vanilla=False)

# Run the experiment and get the history, feature vectors before, and feature vectors after
history, before_vectors, after_vectors = run_experiment(vit_sl, x_train, y_train, x_test, y_test, LEARNING_RATE, WEIGHT_DECAY, BATCH_SIZE, EPOCHS)


Epoch 1/10
9/9 [==============================] - 17s 568ms/step - loss: 6.4358 - accuracy: 0.1826 - top-5-accuracy: 0.2768 - val_loss: 2.8017 - val_accuracy: 1.0000 - val_top-5-accuracy: 1.0000
Epoch 2/10
9/9 [==============================] - 3s 367ms/step - loss: 4.3196 - accuracy: 0.4560 - top-5-accuracy: 0.9483 - val_loss: 3.7950 - val_accuracy: 0.6217 - val_top-5-accuracy: 1.0000
Epoch 3/10
9/9 [==============================] - 3s 368ms/step - loss: 4.1934 - accuracy: 0.4681 - top-5-accuracy: 0.9957 - val_loss: 3.0972 - val_accuracy: 0.9957 - val_top-5-accuracy: 1.0000
Epoch 4/10
9/9 [==============================] - 3s 374ms/step - loss: 4.2155 - accuracy: 0.4222 - top-5-accuracy: 0.9227 - val_loss: 3.1638 - val_accuracy: 1.0000 - val_top-5-accuracy: 1.0000
Epoch 5/10
9/9 [==============================] - 3s 379ms/step - loss: 4.0771 - accuracy: 0.4807 - top-5-accuracy: 0.9928 - val_loss: 3.2289 - val_accuracy: 0.0783 - val_top-5-accuracy: 1.0000
Epoch 6/10
9/9 [=============

In [38]:
# Print the shape of the feature vectors
print("Shape of feature vectors:", before_vectors.shape)

# Print the length of the feature vectors
print("Length of feature vectors:", len(before_vectors))

# Print the first 10 values of the feature vectors
print("First 1 values of feature vectors:")
print(before_vectors[:1])

Shape of feature vectors: (2300, 1024)
Length of feature vectors: 2300
First 1 values of feature vectors:
[[ 3.5828161   3.51928    -0.16993013 ... -0.16995241 -0.16986436
  -0.16905013]]


In [39]:
# Print the shape of the feature vectors
print("Shape of feature vectors:", after_vectors.shape)

# Print the length of the feature vectors
print("Length of feature vectors:", len(after_vectors))

# Print the first 10 values of the feature vectors
print("First 1 values of feature vectors:")
print(after_vectors[:1])

Shape of feature vectors: (2300, 1024)
Length of feature vectors: 2300
First 1 values of feature vectors:
[[ 3.5828161   3.51928    -0.16993013 ... -0.16995241 -0.16986436
  -0.16905013]]


In [40]:

# Access the features tensor
features = vit_sl.layers[-2].output

# Check the size of the feature vector
feature_vector_size = features.shape
print("Size of the feature vector:", feature_vector_size)
# Assuming `feature` is the variable you want to check

if isinstance(features, list):
    print("The features is a list.")
elif isinstance(features, np.ndarray):
    print("The features is an array.")
else:
    print("The features is an object.")

Size of the feature vector: (None, 1024)
The features is an object.


In [56]:
len(y_test)

667

In [72]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, concatenate



# Define input layers for image and text features
image_input = Input(shape=(1024,))
text_input = Input(shape=(1024,))

# Concatenate the input layers
concatenated = concatenate([image_input, text_input])

# Add fully connected layers for classification
dense1 = Dense(512, activation='relu')(concatenated)
dense2 = Dense(256, activation='relu')(dense1)
output = Dense(1, activation='sigmoid')(dense2)

# Define the multimodal model
model = Model(inputs=[image_input, text_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the multimodal model

model.fit([after_vectors, bert2], y_train, epochs=10, batch_size=32)

# # Test the multimodal model
# test_loss, test_acc = model.evaluate([x_test], y_test)
# print('Test accuracy:', test_acc)


Epoch 1/10
72/72 [==============================] - 2s 4ms/step - loss: 0.4067 - accuracy: 0.8152
Epoch 2/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3846 - accuracy: 0.8339
Epoch 3/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3752 - accuracy: 0.8374
Epoch 4/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3725 - accuracy: 0.8357
Epoch 5/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3656 - accuracy: 0.8409
Epoch 6/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3666 - accuracy: 0.8378
Epoch 7/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3844 - accuracy: 0.8370
Epoch 8/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3637 - accuracy: 0.8422
Epoch 9/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3768 - accuracy: 0.8370
Epoch 10/10
72/72 [==============================] - 0s 5ms/step - loss: 0.3688 - accuracy: 0.8383


In [73]:
import numpy as np

# Get the feature vectors for images and text
# image_feature_vectors = # Shape: (2300, 1024)
# text_feature_vectors = # Shape: (2300, 1024)

# Select the relevant samples for testing
image_test_feature_vectors = after_vectors[:667]
text_test_feature_vectors = bert_test[:667]

# Evaluate the multimodal model
test_loss, test_acc = model.evaluate({'image_inputs': image_test_feature_vectors, 'text_inputs': text_test_feature_vectors}, y_test)

print('Test accuracy:', test_acc)


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "/opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/input_spec.py", line 197, in assert_input_compatibility
        raise ValueError(

    ValueError: Missing data for input "inpu

In [65]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Image feature vector shape: (2300, 1024)
# Text feature vector shape: (2300, 1024)
# Test word embedding shape: (667, 1024)

# Define the multimodal model
image_inputs = keras.Input(shape=(1024,), name='image_inputs')
text_inputs = keras.Input(shape=(1024,), name='text_inputs')

# Concatenate the image and text feature vectors
concatenated = layers.concatenate([image_inputs, text_inputs])

# Add more layers for multimodal classification
dense1 = layers.Dense(256, activation='relu')(concatenated)
dense2 = layers.Dense(128, activation='relu')(dense1)
dropout1 = layers.Dropout(0.1)(dense2)
dense3 = layers.Dense(64, activation='relu')(dropout1)
dropout2 = layers.Dropout(0.2)(dense3)
outputs = layers.Dense(1, activation='softmax')(dropout2)

# Create the multimodal model
model = keras.Model(inputs=[image_inputs, text_inputs], outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the multimodal model
history = model.fit({'image_inputs': after_vectors, 'text_inputs': bert2}, y_train, epochs=10, batch_size=32)




Epoch 1/10
72/72 [==============================] - 2s 4ms/step - loss: 0.4132 - accuracy: 0.5574
Epoch 2/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3904 - accuracy: 0.5574
Epoch 3/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3802 - accuracy: 0.5574
Epoch 4/10
72/72 [==============================] - 0s 5ms/step - loss: 0.3804 - accuracy: 0.5574
Epoch 5/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3695 - accuracy: 0.5574
Epoch 6/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3731 - accuracy: 0.5574
Epoch 7/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3833 - accuracy: 0.5574
Epoch 8/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3720 - accuracy: 0.5574
Epoch 9/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3823 - accuracy: 0.5574
Epoch 10/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3697 - accuracy: 0.5574


In [71]:
import numpy as np

# Get the feature vectors for images and text
# image_feature_vectors = # Shape: (2300, 1024)
# text_feature_vectors = # Shape: (2300, 1024)

# Select the relevant samples for testing
image_test_feature_vectors = after_vectors[:667]
text_test_feature_vectors = bert_test[:667]

# Evaluate the multimodal model
test_loss, test_acc = model.evaluate({'image_inputs': image_test_feature_vectors, 'text_inputs': text_test_feature_vectors}, y_test)

print('Test accuracy:', test_acc)


21/21 [==============================] - 0s 3ms/step - loss: 0.6837 - accuracy: 0.4078
Test accuracy: 0.40779611468315125
